# Reference Forecasts

This [Jupyter notebook](https://jupyter.org) is designed to introduce you to the [Solar Forecast Arbiter's](https://solarforecastarbiter.org/) built-in reference forecast capabilities. It is divided into 2 sections:

1. [NWP-based forecasts](#NWP-based-forecasts)
2. [Persistence forecasts](#persistence-forecasts)

The API documentation is available [here](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html).

In [1]:
import datetime
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd

from bokeh.core.properties import value
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.palettes import Category10_10 as PALETTE
TOOLS = "pan,box_zoom,xwheel_zoom,reset,save,box_select"
output_notebook()

Loading BokehJS ...

In [2]:
from solarforecastarbiter import datamodel

## NWP-based forecasts

Forecasts based on NWP model data are used for intraday and longer forecasts. The Solar Forecast Arbiter contains a set of functions to process data from NWP forecasts. Here, we explore some of the functionality with an emphasis on obtaining results. See the [NWP section of the documentation](https://solarforecastarbiter-core.readthedocs.io/en/latest/reference_forecasts.html#nwp) for additional information.

The `solarforecastarbiter-core` package includes a handful of subsetted NWP model runs for testing. We'll use these for our demonstration below. The models were all initialized at 2019-05-15 00Z and the subsets include about half a degree of latitude and longitude near Tucson, AZ.

In [3]:
from solarforecastarbiter.io import nwp
# find the files
base_path = Path(nwp.__file__).resolve().parents[0] / 'tests/data'
# define file loading function that knows where to find the files
load_forecast = partial(nwp.load_forecast, base_path=base_path)

In [4]:
# define coordinates
latitude = 32.2
longitude = -110.9
elevation = 700

# define initialization time, forecast start time, forecast end time
init_time = pd.Timestamp('20190515T0000Z')
start = pd.Timestamp('20190515T0100Z')
end = pd.Timestamp('20190518T0000Z')

In [5]:
from solarforecastarbiter.reference_forecasts import main, models

In [6]:
# define a Site object
site = datamodel.Site(
    name='Tucson, AZ',
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    timezone='America/Phoenix'
)

Next we select a NWP+post processing model, load and process the data, and plot the processed data.

Here we use the subhourly irradiance directly from the HRRR model.

In [7]:
# select the model
model = models.hrrr_subhourly_to_subhourly_instantaneous

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('GHI', 'DNI', 'DHI')):
    fig.line(irrad.index, irrad, legend=name, color=next(palette), line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
fig.legend.location = "top_left"
show(fig)

The Solar Forecast Arbiter includes processing functions derive GHI from cloud cover and then compute DNI and DHI using the Erbs model. These functions are applied to data from the GFS, NAM, and RAP weather models.

In [8]:
# select the model
model = models.gfs_quarter_deg_to_hourly_mean

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(site, model_wrapped, init_time, start, end)

fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('GHI', 'DNI', 'DHI')):
    fig.line(irrad.index, irrad, legend=name, color=next(palette), line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
fig.legend.location = "top_left"
show(fig)

In [9]:
# define the modeling parameters
modeling_parameters = datamodel.SingleAxisModelingParameters(
    ac_capacity=10,
    dc_capacity=13,
    temperature_coefficient=-0.003,
    dc_loss_factor=0,
    ac_loss_factor=0,
    axis_tilt=0,
    axis_azimuth=0,
    ground_coverage_ratio=0.4,
    backtrack=True,
    max_rotation_angle=50,
)
# define a SolarPowerPlant object
plant = datamodel.SolarPowerPlant(
    name='Tucson AZ Plant',
    latitude=latitude,
    longitude=longitude,
    elevation=elevation,
    timezone='America/Phoenix',
    modeling_parameters=modeling_parameters
)

In [10]:
# select the model
model = models.gfs_quarter_deg_to_hourly_mean

# tell the model where to find the NWP data
model_wrapped = partial(model, load_forecast=load_forecast)

# load and process the NWP data
ghi, dni, dhi, air_temperature, wind_speed, ac_power = main.run(plant, model_wrapped, init_time, start, end)

# plotting details...
data = dict(zip(('GHI', 'DNI', 'DHI', 'Air temperature', 'Wind speed', 'AC power', 'index'),
                (ghi, dni, dhi, air_temperature, wind_speed, ac_power, ac_power.index)))
source = ColumnDataSource(data)

fig_kwargs = dict(tools=TOOLS, x_axis_type="datetime", plot_height=200)
palette = iter(PALETTE)
fig1 = figure(**fig_kwargs)
for name in ('GHI', 'DNI', 'DHI'):
    fig1.line(x='index', y=name, source=source, legend=value(name), color=next(palette), line_width=2)
fig1.yaxis.axis_label = "Irradiance (W/m^2)"
fig1.legend.location = "top_left"

fig_kwargs['x_range'] = fig1.x_range  # link x-zoom
fig2 = figure(**fig_kwargs)
fig2.line(x='index', y='Air temperature', source=source, legend=value("Air temperature"), color=next(palette), line_width=2)
fig2.yaxis.axis_label = "Air temperature (C)"
fig2.legend.location = "top_left"

fig3 = figure(**fig_kwargs)
fig3.line(x='index', y='Wind speed', source=source, legend=value("Wind speed"), color=next(palette), line_width=2)
fig3.yaxis.axis_label = "Wind speed (m/s)"
fig3.legend.location = "top_left"

fig4 = figure(**fig_kwargs)
fig4.line(x='index', y='AC power', source=source, legend=value("AC power"), color=next(palette), line_width=2)
fig4.yaxis.axis_label = "AC power (MW)"
fig4.xaxis.axis_label = 'Time (UTC)'
fig4.legend.location = "top_left"

grid = gridplot([fig1, fig2, fig3, fig4], ncols=1, plot_width=800)

show(grid)

In [11]:
# returns tuple of ghi, dni, dhi, air temperature, wind speed, resampling function, solar position function
ghi, dni, dhi, air_temperature, wind_speed, resampler, solpos_func = models.hrrr_subhourly_to_subhourly_instantaneous(
    latitude, longitude, elevation, init_time, start, end, load_forecast=load_forecast)

Plot the irradiance components of `out`.

In [12]:
fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('ghi', 'dni', 'dhi')):
    resampled = resampler(irrad)
    fig.line(resampled.index, resampled, legend=name, color=next(palette), line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
show(fig)

In [13]:
ghi, dni, dhi, air_temperature, wind_speed, resampler, solpos_func = models.gfs_quarter_deg_hourly_to_hourly_mean(
    latitude, longitude, elevation, init_time, start, end, load_forecast=load_forecast)

fig = figure(title="Irradiance (W/m^2)", tools=TOOLS, x_axis_type="datetime", plot_width=800, plot_height=400)
palette = iter(PALETTE)
for irrad, name in zip((ghi, dni, dhi), ('ghi', 'dni', 'dhi')):
    resampled = resampler(irrad)
    color = next(palette)
    fig.line(resampled.index, resampled, legend=name, color=color, line_width=2)
    fig.line(irrad.index, irrad, legend=name, color=color, line_width=2)
fig.xaxis.axis_label = 'Time (UTC)'
show(fig)

In [14]:
forecast = datamodel.Forecast(
    name='NWP fx', 
    issue_time_of_day=datetime.time(0),  # 0Z
    lead_time_to_start=pd.Timedelta('1hr'),
    interval_length=pd.Timedelta('1hr'),
    run_length=pd.Timedelta('24hr'),
    interval_label='ending',
    interval_value_type='mean',
    variable='ghi',
    site=site
)

## Persistence forecasts

In [15]:
from solarforecastarbiter.reference_forecasts import persistence